In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Balls_Lottery_26_11_2022.csv to Balls_Lottery_26_11_2022 (1).csv


In [3]:
import io
df = pd.read_csv(io.BytesIO(uploaded['Balls_Lottery_26_11_2022.csv']))
# Dataset is now stored in a Pandas Dataframe

In [4]:
print(df.shape)
print(list(df.columns))

(1297, 8)
['Date', 'Ball_1', 'Ball_2', 'Ball_3', 'Ball_4', 'Ball_5', 'Ball_6', 'Bonus_Ball']


In [5]:
df.head()

,Date,Ball_1,Ball_2,Ball_3,Ball_4,Ball_5,Ball_6,Bonus_Ball
0,05/03/2011,2,17,19,21,26,31,6
1,08/03/2011,2,7,8,9,13,28,3
2,12/03/2011,3,7,14,17,18,20,1
3,15/03/2011,4,11,18,19,22,28,2
4,19/03/2011,8,10,17,19,24,32,3


In [6]:
# Amount of games we need to take into consideration for prediction
window_length = 7
window_length

7

In [7]:
df1 = df.copy()
df.drop(['Date'], axis=1, inplace=True)

number_of_features = df.shape[1] 

In [8]:
train = df.copy()
train.head((window_length+1))

,Ball_1,Ball_2,Ball_3,Ball_4,Ball_5,Ball_6,Bonus_Ball
0,2,17,19,21,26,31,6
1,2,7,8,9,13,28,3
2,3,7,14,17,18,20,1
3,4,11,18,19,22,28,2
4,8,10,17,19,24,32,3
5,4,5,7,8,15,33,7
6,8,12,13,19,23,29,3
7,6,13,21,23,24,29,1


In [9]:
train_rows = train.values.shape[0]
train_samples = np.empty([ train_rows - window_length, window_length, number_of_features], dtype=float)
train_labels = np.empty([ train_rows - window_length, number_of_features], dtype=float)
for i in range(0, train_rows-window_length):
    train_samples[i] = train.iloc[i : i+window_length, 0 : number_of_features]
    train_labels[i] = train.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [10]:
train_samples[0]

array([[ 2., 17., 19., 21., 26., 31.,  6.],
       [ 2.,  7.,  8.,  9., 13., 28.,  3.],
       [ 3.,  7., 14., 17., 18., 20.,  1.],
       [ 4., 11., 18., 19., 22., 28.,  2.],
       [ 8., 10., 17., 19., 24., 32.,  3.],
       [ 4.,  5.,  7.,  8., 15., 33.,  7.],
       [ 8., 12., 13., 19., 23., 29.,  3.]])

In [11]:
train_labels[0]

array([ 6., 13., 21., 23., 24., 29.,  1.])

In [12]:
scaler = StandardScaler()
transformed_dataset = scaler.fit_transform(train.values)
scaled_train_samples = pd.DataFrame(data=transformed_dataset, index=train.index)

In [13]:
scaled_train_samples.head(window_length+1)

,0,1,2,3,4,5,6
0,-0.803064,1.114072,0.453402,-0.090992,-0.205642,-0.378622,1.002704
1,-0.803064,-0.714055,-1.341958,-2.074474,-2.601857,-1.091355,-0.499036
2,-0.569370,-0.714055,-0.362671,-0.752153,-1.680236,-2.991977,-1.500196
3,-0.335676,0.017196,0.290187,-0.421573,-0.942939,-1.091355,-0.999616
4,0.599100,-0.165617,0.126973,-0.421573,-0.574290,-0.141045,-0.499036
5,-0.335676,-1.079680,-1.505173,-2.239764,-2.233208,0.096533,1.503283
6,0.599100,0.200009,-0.525885,-0.421573,-0.758614,-0.853778,-0.499036
7,0.131712,0.382821,0.779831,0.239588,-0.574290,-0.853778,-1.500196


In [14]:
x_train = np.empty([ train_rows - window_length, window_length, number_of_features], dtype=float)
y_train = np.empty([ train_rows - window_length, number_of_features], dtype=float)

for i in range(0, train_rows-window_length):
    x_train[i] = scaled_train_samples.iloc[i : i+window_length, 0 : number_of_features]
    y_train[i] = scaled_train_samples.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mse

In [16]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [17]:
# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a third Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(37))
# Adding the last output layer
model.add(Dense(number_of_features))

In [18]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

In [19]:
model.fit(x=x_train, y=y_train, batch_size=100, epochs=1200, verbose=2)

Epoch 1/1200
13/13 - 18s - loss: 0.9994 - accuracy: 0.1884 - 18s/epoch - 1s/step
Epoch 2/1200
13/13 - 0s - loss: 0.9961 - accuracy: 0.1752 - 174ms/epoch - 13ms/step
Epoch 3/1200
13/13 - 0s - loss: 0.9942 - accuracy: 0.1589 - 166ms/epoch - 13ms/step
Epoch 4/1200
13/13 - 0s - loss: 0.9929 - accuracy: 0.1558 - 175ms/epoch - 13ms/step
Epoch 5/1200
13/13 - 0s - loss: 0.9923 - accuracy: 0.1543 - 168ms/epoch - 13ms/step
Epoch 6/1200
13/13 - 0s - loss: 0.9921 - accuracy: 0.1705 - 172ms/epoch - 13ms/step
Epoch 7/1200
13/13 - 0s - loss: 0.9916 - accuracy: 0.1729 - 167ms/epoch - 13ms/step
Epoch 8/1200
13/13 - 0s - loss: 0.9906 - accuracy: 0.1752 - 182ms/epoch - 14ms/step
Epoch 9/1200
13/13 - 0s - loss: 0.9896 - accuracy: 0.1752 - 181ms/epoch - 14ms/step
Epoch 10/1200
13/13 - 0s - loss: 0.9901 - accuracy: 0.1690 - 180ms/epoch - 14ms/step
Epoch 11/1200
13/13 - 0s - loss: 0.9891 - accuracy: 0.1752 - 180ms/epoch - 14ms/step
Epoch 12/1200
13/13 - 0s - loss: 0.9896 - accuracy: 0.1845 - 172ms/epoch - 13

In [20]:
val = df1.copy()
val = val.tail((window_length+window_length))
val = val.head((window_length+1))
val

,Date,Ball_1,Ball_2,Ball_3,Ball_4,Ball_5,Ball_6,Bonus_Ball
1283,18/10/2022,5,7,13,17,25,32,4
1284,22/10/2022,5,11,13,19,32,37,5
1285,25/10/2022,5,7,10,16,31,32,4
1286,27/10/2022,12,13,22,25,28,36,2
1287,29/10/2022,2,5,15,24,32,33,3
1288,01/11/2022,2,3,4,5,11,15,3
1289,05/11/2022,6,9,22,25,31,34,4
1290,08/11/2022,4,5,6,31,33,36,7


In [21]:
val_Date = df1.iloc[ (val.tail().index[-1]) ]['Date']
val1 = val.head((window_length))
val1

,Date,Ball_1,Ball_2,Ball_3,Ball_4,Ball_5,Ball_6,Bonus_Ball
1283,18/10/2022,5,7,13,17,25,32,4
1284,22/10/2022,5,11,13,19,32,37,5
1285,25/10/2022,5,7,10,16,31,32,4
1286,27/10/2022,12,13,22,25,28,36,2
1287,29/10/2022,2,5,15,24,32,33,3
1288,01/11/2022,2,3,4,5,11,15,3
1289,05/11/2022,6,9,22,25,31,34,4


In [22]:
val1.drop(['Date'], axis=1, inplace=True)
val1 = np.array(val1)
val1

<ipython-input-22-0cea31e8c13c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val1.drop(['Date'], axis=1, inplace=True)


array([[ 5,  7, 13, 17, 25, 32,  4],
       [ 5, 11, 13, 19, 32, 37,  5],
       [ 5,  7, 10, 16, 31, 32,  4],
       [12, 13, 22, 25, 28, 36,  2],
       [ 2,  5, 15, 24, 32, 33,  3],
       [ 2,  3,  4,  5, 11, 15,  3],
       [ 6,  9, 22, 25, 31, 34,  4]])

In [23]:
x_val = scaler.transform(val1)
x_val

array([[-1.01982055e-01, -7.14054948e-01, -5.25885499e-01,
        -7.52152915e-01, -3.89965848e-01, -1.41044586e-01,
         1.54380836e-03],
       [-1.01982055e-01,  1.71959541e-02, -5.25885499e-01,
        -4.21572661e-01,  9.00303806e-01,  1.04684391e+00,
         5.02123667e-01],
       [-1.01982055e-01, -7.14054948e-01, -1.01552907e+00,
        -9.17443042e-01,  7.15979570e-01, -1.41044586e-01,
         1.54380836e-03],
       [ 1.53387497e+00,  3.82821405e-01,  9.43045209e-01,
         5.70168103e-01,  1.63006861e-01,  8.09266209e-01,
        -9.99615910e-01],
       [-8.03063637e-01, -1.07968040e+00, -1.99456453e-01,
         4.04877975e-01,  9.00303806e-01,  9.65331128e-02,
        -4.99036051e-01],
       [-8.03063637e-01, -1.44530585e+00, -1.99481621e+00,
        -2.73563444e+00, -2.97050516e+00, -4.17986547e+00,
        -4.99036051e-01],
       [ 1.31711806e-01, -3.48429497e-01,  9.43045209e-01,
         5.70168103e-01,  7.15979570e-01,  3.34110812e-01,
         1.5438083

In [24]:
y_val_pred = model.predict(np.array([x_val]))
print("The predicted numbers for the lottery game which took place on",val_Date, "were (without rounding up):", scaler.inverse_transform(y_val_pred).astype(int)[0])

1/1 [==============================] - 3s 3s/step
The predicted numbers for the lottery game which took place on 08/11/2022 were (without rounding up): [ 3  4  6 30 33 36  6]


In [25]:
y_val_pred = model.predict(np.array([x_val]))
print("The predicted numbers for the lottery game which took place on",val_Date, "were (with rounding up):", scaler.inverse_transform(y_val_pred).astype(int)[0]+1)

1/1 [==============================] - 0s 30ms/step
The predicted numbers for the lottery game which took place on 08/11/2022 were (with rounding up): [ 4  5  7 31 34 37  7]


In [26]:
print("The actual numbers for the lottery game which took place on",val_Date, "were:")
y_val_true = val.tail(1)
y_val_true

The actual numbers for the lottery game which took place on 08/11/2022 were:


,Date,Ball_1,Ball_2,Ball_3,Ball_4,Ball_5,Ball_6,Bonus_Ball
1290,08/11/2022,4,5,6,31,33,36,7


In [27]:
test = df1.copy()
test = test.tail((window_length+1))
test

,Date,Ball_1,Ball_2,Ball_3,Ball_4,Ball_5,Ball_6,Bonus_Ball
1289,05/11/2022,6,9,22,25,31,34,4
1290,08/11/2022,4,5,6,31,33,36,7
1291,12/11/2022,4,9,17,25,33,37,6
1292,15/11/2022,4,9,10,15,26,27,2
1293,17/11/2022,3,12,13,16,29,34,1
1294,19/11/2022,10,13,16,29,30,35,4
1295,22/11/2022,11,17,25,26,27,32,4
1296,26/11/2022,2,5,9,12,15,16,4


In [28]:
test_Date = df1.iloc[ (test.tail().index[-1]) ]['Date']
test1 = test.head(window_length)
test1

,Date,Ball_1,Ball_2,Ball_3,Ball_4,Ball_5,Ball_6,Bonus_Ball
1289,05/11/2022,6,9,22,25,31,34,4
1290,08/11/2022,4,5,6,31,33,36,7
1291,12/11/2022,4,9,17,25,33,37,6
1292,15/11/2022,4,9,10,15,26,27,2
1293,17/11/2022,3,12,13,16,29,34,1
1294,19/11/2022,10,13,16,29,30,35,4
1295,22/11/2022,11,17,25,26,27,32,4


In [29]:
test1.drop(['Date'], axis=1, inplace=True)
test1 = np.array(test1)
test1

<ipython-input-29-cd1599982823>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test1.drop(['Date'], axis=1, inplace=True)


array([[ 6,  9, 22, 25, 31, 34,  4],
       [ 4,  5,  6, 31, 33, 36,  7],
       [ 4,  9, 17, 25, 33, 37,  6],
       [ 4,  9, 10, 15, 26, 27,  2],
       [ 3, 12, 13, 16, 29, 34,  1],
       [10, 13, 16, 29, 30, 35,  4],
       [11, 17, 25, 26, 27, 32,  4]])

In [30]:
x_test = scaler.transform(test1)
x_test

array([[ 1.31711806e-01, -3.48429497e-01,  9.43045209e-01,
         5.70168103e-01,  7.15979570e-01,  3.34110812e-01,
         1.54380836e-03],
       [-3.35675915e-01, -1.07968040e+00, -1.66838716e+00,
         1.56190887e+00,  1.08462804e+00,  8.09266209e-01,
         1.50328339e+00],
       [-3.35675915e-01, -3.48429497e-01,  1.26972594e-01,
         5.70168103e-01,  1.08462804e+00,  1.04684391e+00,
         1.00270353e+00],
       [-3.35675915e-01, -3.48429497e-01, -1.01552907e+00,
        -1.08273317e+00, -2.05641611e-01, -1.32893308e+00,
        -9.99615910e-01],
       [-5.69369776e-01,  2.00008680e-01, -5.25885499e-01,
        -9.17443042e-01,  3.47331097e-01,  3.34110812e-01,
        -1.50019577e+00],
       [ 1.06648725e+00,  3.82821405e-01, -3.62419296e-02,
         1.23132861e+00,  5.31655334e-01,  5.71688510e-01,
         1.54380836e-03],
       [ 1.30018111e+00,  1.11407231e+00,  1.43268878e+00,
         7.35458230e-01, -2.13173751e-02, -1.41044586e-01,
         1.5438083

In [31]:
y_test_pred = model.predict(np.array([x_test]))
print("The predicted numbers for the lottery game which took place on",test_Date, "were (without rounding up):", scaler.inverse_transform(y_test_pred).astype(int)[0])

1/1 [==============================] - 0s 32ms/step
The predicted numbers for the lottery game which took place on 26/11/2022 were (without rounding up): [ 2  5  8 12 14 16  4]


In [32]:
y_test_pred = model.predict(np.array([x_test]))
print("The predicted numbers for the lottery game which took place on",test_Date, "were (with rounding up):", scaler.inverse_transform(y_test_pred).astype(int)[0]+1)

1/1 [==============================] - 0s 29ms/step
The predicted numbers for the lottery game which took place on 26/11/2022 were (with rounding up): [ 3  6  9 13 15 17  5]


In [33]:
print("The actual numbers for the lottery game which took place on",test_Date, "were:")
y_test_true = test.tail(1)
y_test_true

The actual numbers for the lottery game which took place on 26/11/2022 were:


,Date,Ball_1,Ball_2,Ball_3,Ball_4,Ball_5,Ball_6,Bonus_Ball
1296,26/11/2022,2,5,9,12,15,16,4


In [34]:
next = df.copy()
next = next.tail((window_length))
next

,Ball_1,Ball_2,Ball_3,Ball_4,Ball_5,Ball_6,Bonus_Ball
1290,4,5,6,31,33,36,7
1291,4,9,17,25,33,37,6
1292,4,9,10,15,26,27,2
1293,3,12,13,16,29,34,1
1294,10,13,16,29,30,35,4
1295,11,17,25,26,27,32,4
1296,2,5,9,12,15,16,4


In [35]:
next_Date = '29/11/2022'

next = np.array(next)
next

array([[ 4,  5,  6, 31, 33, 36,  7],
       [ 4,  9, 17, 25, 33, 37,  6],
       [ 4,  9, 10, 15, 26, 27,  2],
       [ 3, 12, 13, 16, 29, 34,  1],
       [10, 13, 16, 29, 30, 35,  4],
       [11, 17, 25, 26, 27, 32,  4],
       [ 2,  5,  9, 12, 15, 16,  4]])

In [36]:
x_next = scaler.transform(next)
x_next

array([[-3.35675915e-01, -1.07968040e+00, -1.66838716e+00,
         1.56190887e+00,  1.08462804e+00,  8.09266209e-01,
         1.50328339e+00],
       [-3.35675915e-01, -3.48429497e-01,  1.26972594e-01,
         5.70168103e-01,  1.08462804e+00,  1.04684391e+00,
         1.00270353e+00],
       [-3.35675915e-01, -3.48429497e-01, -1.01552907e+00,
        -1.08273317e+00, -2.05641611e-01, -1.32893308e+00,
        -9.99615910e-01],
       [-5.69369776e-01,  2.00008680e-01, -5.25885499e-01,
        -9.17443042e-01,  3.47331097e-01,  3.34110812e-01,
        -1.50019577e+00],
       [ 1.06648725e+00,  3.82821405e-01, -3.62419296e-02,
         1.23132861e+00,  5.31655334e-01,  5.71688510e-01,
         1.54380836e-03],
       [ 1.30018111e+00,  1.11407231e+00,  1.43268878e+00,
         7.35458230e-01, -2.13173751e-02, -1.41044586e-01,
         1.54380836e-03],
       [-8.03063637e-01, -1.07968040e+00, -1.17874359e+00,
        -1.57860355e+00, -2.23320821e+00, -3.94228777e+00,
         1.5438083

In [37]:
y_next_pred = model.predict(np.array([x_next]))
print("The predicted numbers for the lottery game which will take place on",next_Date, "are (without rounding up):", scaler.inverse_transform(y_next_pred).astype(int)[0])

1/1 [==============================] - 0s 34ms/step
The predicted numbers for the lottery game which will take place on 29/11/2022 are (without rounding up): [ 2  8 12 13 16 21  4]


In [38]:
y_next_pred = model.predict(np.array([x_next]))
print("The predicted numbers for the lottery game which will take place on",next_Date, "are (with rounding up):", scaler.inverse_transform(y_next_pred).astype(int)[0]+1)

1/1 [==============================] - 0s 32ms/step
The predicted numbers for the lottery game which will take place on 29/11/2022 are (with rounding up): [ 3  9 13 14 17 22  5]
